In [41]:
import os
import pathlib
import sklearn
import numpy as np
import pandas as pd
import xgboost as xgb

In [63]:
parent_path = str(pathlib.Path(os.getcwd()).parent)
df = pd.read_csv(os.path.join(parent_path, 'data/data.csv'))
df = df.drop(['Unnamed: 0'], axis=1)
parent_path = str(pathlib.Path(os.getcwd()).parent)
df = pd.read_csv(os.path.join(parent_path, 'data/data.csv'))
df = df.drop(['Unnamed: 0'], axis=1)
df.sample(5)
print(df.shape)
# use a subset of the data
df_small = df.sample(int(len(df)*0.8))
df_small = df_small.drop(df_small[df_small.moneyness>1.7].index)
df_small = df_small.drop(df_small[df_small.days_to_maturity>365].index)
# df_small = df_small.drop(df_small[df_small.contract_price<1].index)
df_small.head()
print(df_small.shape)

(107499, 13)
(85999, 13)


In [64]:
small_dataframe_BS = np.vstack((df_small['strike'].values,
                      df_small['underlyings_price'].values,
                      df_small['days_to_maturity'].values,
                      df_small['volatility'].values,
                      df_small['rate'].values,
                      df_small['contract_price'].values)).T  
N = len(small_dataframe_BS)
indices = np.random.permutation(N)
train_ds, test_ds = np.split(small_dataframe_BS[indices], [int(.8*N)]) 

In [65]:
x_train = train_ds[:, :-1]
y_train = train_ds[:, -1]
x_test = test_ds[:, :-1]
y_test = test_ds[:, -1]

In [56]:
regressor = xgb.XGBRegressor(eval_metric='rmse')

In [46]:
from sklearn.model_selection import GridSearchCV

param_grid = {"max_depth":    [4, 5],
              "n_estimators": [500, 600, 700],
              "learning_rate": [0.01, 0.015]}


search = GridSearchCV(regressor, param_grid, cv=5).fit(x_train, y_train)

print("The best hyperparameters are ",search.best_params_)

The best hyperparameters are  {'learning_rate': 0.015, 'max_depth': 5, 'n_estimators': 700}


In [47]:
regressor=xgb.XGBRegressor(learning_rate = search.best_params_["learning_rate"],
                           n_estimators  = search.best_params_["n_estimators"],
                           max_depth     = search.best_params_["max_depth"],
                           eval_metric='rmse')

regressor.fit(x_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric='rmse', gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.015, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=700, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [66]:
# same in the paper
regressor = xgb.XGBRegressor(learning_rate = 0.3,
                            n_estimators = 700, 
                            max_depth = 20, 
                            eval_metric='rmse')
regressor.fit(x_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric='rmse', gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.3, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=20, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=700, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [67]:
regressor.score(x_test, y_test)

0.8795969691927229